## Python libraries

In [1]:
!pip install tensorflow

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from google.colab import drive
from tensorflow.keras import layers

## Constants

In [3]:
MODEL_ALPHA = 0.35
MODEL_INPUT_WIDTH = 48
MODEL_INPUT_HEIGHT = 48
TFL_MODEL_FILE = "model.tflite"
TFL_MODEL_HEADER_FILE = "model.h"
TF_MODEL = "guard_shack_recognition"

# **Transfer learning with Keras**
### Mount the top-level Google Drive directory

In [4]:
drive.mount('/content/drive')

Mounted at /content/drive


### Prepare the train (80%) and validation (20%) datasets

In [5]:
train_dir = "drive/MyDrive/hackathon/dataset"

ds = tf.keras.utils.image_dataset_from_directory(
  train_dir,
  validation_split=0.2,
  subset="both",
  seed=123,
  interpolation="bilinear",
  image_size=(MODEL_INPUT_WIDTH, MODEL_INPUT_HEIGHT)
  )

train_ds = ds[0]
val_ds   = ds[1]

Found 66 files belonging to 3 classes.
Using 53 files for training.
Using 13 files for validation.


### Get the number of classes from the training dataset

In [6]:
class_names = train_ds.class_names
num_classes = len(class_names)
print(class_names)

['allow', 'call', 'deny']


### Rescale the pixel values from [0, 255] tp [-1, 1]

In [7]:
rescale = tf.keras.layers.Rescaling(1./255, offset= -1)
train_ds = train_ds.map(lambda x, y: (rescale(x), y))
val_ds   = val_ds.map(lambda x, y: (rescale(x), y))

### Import the MobileNet v2 pre-trained model

In [8]:
# https://github.com/keras-team/keras-applications/blob/master/keras_applications/mobilenet_v2.py
base_model = MobileNetV2(input_shape=(MODEL_INPUT_WIDTH, MODEL_INPUT_HEIGHT, 3),
                         include_top=False,
                         weights='imagenet',
                         alpha=0.35)

/tmp/ipython-input-1315049705.py:2: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(input_shape=(MODEL_INPUT_WIDTH, MODEL_INPUT_HEIGHT, 3),


2019640/2019640 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


### Freeze the weights



In [9]:
base_model.trainable = False
feat_extr = base_model

print("num. weights:", len(base_model.weights))
print("num. trainable_weights:", len(base_model.trainable_weights))
print("num. non_trainable_weights:", len(base_model.non_trainable_weights))

num. weights: 260
num. trainable_weights: 0
num. non_trainable_weights: 260


### Augment the input data


In [10]:
augmen = tf.keras.Sequential([
  layers.RandomFlip("horizontal_and_vertical"),
  layers.RandomRotation(0.2),
])

train_ds = train_ds.map(lambda x, y: (augmen(x), y))
val_ds   = val_ds.map(lambda x, y: (augmen(x), y))

### Prepare the classification head

In [11]:
layers = tf.keras.layers
global_avg_layer = layers.GlobalAveragePooling2D()
dense_layer = layers.Dense(num_classes, activation='softmax')

### Finalize the model architecture

In [12]:
x = global_avg_layer(feat_extr.layers[-1].output)
x = layers.Dropout(0.2)(x)
outputs = dense_layer(x)
model = tf.keras.Model(inputs=feat_extr.inputs, outputs=outputs)

### Compile the model with a 0.0005 learning rate

In [13]:
lr = 0.0005
opt_f = tf.keras.optimizers.Adam(learning_rate=lr)
loss_f = tf.losses.SparseCategoricalCrossentropy(from_logits=False)

model.compile(
  optimizer=opt_f,
  loss=loss_f,
  metrics=['accuracy'])

### Model Summary

In [14]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 48, 48, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 24, 24,    │        432 │ input_layer[0][0] │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 24, 24,    │         64 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 24, 24,    │          0 │ bn_Conv1[0][0]    │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 24, 24,    │        144 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 24, 24,    │         64 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 24, 24,    │          0 │ expanded_conv_de… │
│ (ReLU)              │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 24, 24, 8) │        128 │ expanded_conv_de… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 24, 24, 8) │         32 │ expanded_conv_pr… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 24, 24,    │        384 │ expanded_conv_pr… │
│ (Conv2D)            │ 48)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 24, 24,    │        192 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 48)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 24, 24,    │          0 │ block_1_expand_B… │
│ (ReLU)              │ 48)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 25, 25,    │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 48)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 12, 12,    │        432 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 48)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 12, 12,    │        192 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 48)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 12, 12,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 48)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 12, 12, 8) │        384 │ block_1_depthwis

 Total params: 414,051 (1.58 MB)

 Trainable params: 3,843 (15.01 KB)

 Non-trainable params: 410,208 (1.56 MB)

### Train the model with 10 epochs

In [15]:
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=10
)

Epoch 1/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 15s 5s/step - accuracy: 0.1757 - loss: 2.0229 - val_accuracy: 0.3846 - val_loss: 1.5524
Epoch 2/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 417ms/step - accuracy: 0.2594 - loss: 1.4252 - val_accuracy: 0.2308 - val_loss: 1.5032
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 455ms/step - accuracy: 0.4456 - loss: 1.1877 - val_accuracy: 0.3077 - val_loss: 1.6567
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 310ms/step - accuracy: 0.2868 - loss: 1.4958 - val_accuracy: 0.4615 - val_loss: 1.1700
Epoch 5/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 330ms/step - accuracy: 0.4226 - loss: 1.1903 - val_accuracy: 0.3077 - val_loss: 1.2574
Epoch 6/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 293ms/step - accuracy: 0.4855 - loss: 0.9565 - val_accuracy: 0.6154 - val_loss: 1.1125
Epoch 7/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 544ms/step - accuracy: 0.5774 - loss: 0.8505 - val_accuracy: 0.2308 - val_loss: 1.2354
Epoch 8/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 384ms/step - accuracy: 0.4833 - loss: 1.1227 - val_accuracy: 0.6923 - val_loss: 0

### Export the TensorFlow model

In [16]:
model.export(TF_MODEL)

Saved artifact at 'guard_shack_recognition'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): List[TensorSpec(shape=(None, 48, 48, 3), dtype=tf.float32, name='keras_tensor')]
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  136247596343056: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136247591556048: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136247591555664: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136247591555472: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136247591553552: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136247591556624: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136247591557200: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136247591556240: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136247591554128: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136247591557008: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136

##**Quantizing and testing the trained model with TensorFlow Lite**
### Acquire new images for the test dataset


In [17]:

# CHANGE TO DESIRED TEST DATASET FOLDER
test_dir = "drive/MyDrive/hackathon/test"

### Rescale the pixel values from [0, 255] to [-1, 1]

In [18]:
test_ds = tf.keras.utils.image_dataset_from_directory(test_dir,
                                                      interpolation="bilinear",
                                                      image_size=(MODEL_INPUT_WIDTH, MODEL_INPUT_HEIGHT))
test_ds  = test_ds.map(lambda x, y: (rescale(x), y))

Found 30 files belonging to 3 classes.


### Quantize the TensorFlow model with the TensorFlow Lite converter

In [19]:
repr_ds = test_ds.unbatch()

def representative_data_gen():
  for i_value, o_value in repr_ds.batch(1).take(48):
    yield [i_value]

converter = tf.lite.TFLiteConverter.from_saved_model(TF_MODEL)
converter.representative_dataset = tf.lite.RepresentativeDataset(representative_data_gen)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8

tfl_model = converter.convert()

### Initialize the TensorFlow Lite interpreter

In [20]:
# Initialize the TFLite interpreter
interp = tf.lite.Interpreter(model_content=tfl_model)

/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


### Allocate the tensor and get input quantization parameters

In [21]:
# Allocate the tensors
interp.allocate_tensors()

# Get input/output layer information
i_details = interp.get_input_details()[0]
o_details = interp.get_output_details()[0]

# Get input quantization parameters.
i_quant = i_details["quantization_parameters"]
i_scale      = i_quant['scales'][0]
i_zero_point = i_quant['zero_points'][0]

### Evaluate the accuracy of the quantized TensorFlow Lite model

In [22]:
test_ds0 = test_ds.unbatch()

num_correct_samples = 0
num_samples   = len(list(test_ds0.batch(1)))

for i_value, o_value in test_ds0.batch(1):
  i_value = (i_value / i_scale) + i_zero_point
  i_value = tf.cast(i_value, dtype=tf.int8)
  interp.set_tensor(i_details["index"], i_value)
  interp.invoke()
  o_pred = interp.get_tensor(o_details["index"])[0]

  if np.argmax(o_pred) == o_value:
    num_correct_samples += 1

print("Accuracy:", num_correct_samples/num_samples)

Accuracy: 0.4


### Convert the TensorFlow model to C-byte array with xxd

In [23]:
open(TFL_MODEL_FILE, "wb").write(tfl_model)

619840

In [24]:
!apt-get update && apt-get -qq install xxd
# Convert the TFLite model to a C source file
!xxd -c 60 -i {TFL_MODEL_FILE} > {TFL_MODEL_HEADER_FILE}
!sed -i 's/unsigned char/const unsigned char/g' $TFL_MODEL_HEADER_FILE
!sed -i 's/const/alignas(8) const/g' $TFL_MODEL_HEADER_FILE

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:4 https://cli.github.com/packages stable InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,123 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,526 kB]
Get:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,825 kB]
Hit:13 https://ppa.launchpadcontent.net/graphics-drivers

### Get the TensorFlow model size in bytes to estimate the program memory usage

In [25]:
size_tfl_model = len(tfl_model)
print(len(tfl_model), "bytes")

619840 bytes


### Convert the Model to TFLite to be used with Microcontrollers

In [26]:
# Convert the TFLite model to a C source file
!xxd -i {TFL_MODEL_FILE} > {TFL_MODEL_HEADER_FILE}

# Update the header file name to be a valid C variable name
REPLACEMENT_TEXT = TFL_MODEL_FILE.replace('.', '_')
!sed -i 's/{REPLACEMENT_TEXT}/{TF_MODEL}/g' {TFL_MODEL_HEADER_FILE}

print(f"TensorFlow Lite model C header file saved to {TFL_MODEL_HEADER_FILE}")

TensorFlow Lite model C header file saved to model.h


In [27]:
converter = tf.lite.TFLiteConverter.from_saved_model(TF_MODEL)
tflite_model = converter.convert()

with open(TFL_MODEL_FILE, 'wb') as f:
    f.write(tflite_model)

print(f"TensorFlow Lite model saved to {TFL_MODEL_FILE}")

TensorFlow Lite model saved to model.tflite
